# 위치 정보 시각화

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

In [6]:
# 서울시 자치구 위도, 경도 정보
location = pd.read_csv("./data/seoul_district.csv")
location.head()

,code,area,lon,lat
0,11110,종로구,126.977321,37.594917
1,11140,중구,126.995968,37.560144
2,11170,용산구,126.979907,37.531385
3,11200,성동구,127.041059,37.551030
4,11215,광진구,127.085744,37.546706


In [9]:
for i, row in location.iterrows():
    print(i, row['area'])

0 종로구
1 중구
2 용산구
3 성동구
4 광진구
5 동대문구
6 중랑구
7 성북구
8 강북구
9 도봉구
10 노원구
11 은평구
12 서대문구
13 마포구
14 양천구
15 강서구
16 구로구
17 금천구
18 영등포구
19 동작구
20 관악구
21 서초구
22 강남구
23 송파구
24 강동구


In [85]:
import folium

# 초기 맵 설정
m = folium.Map(location=[location['lat'].mean(), location['lon'].mean()], zoom_start=11)

# 데이터 포인트 추가
for i, row in location.iterrows():
    folium.Marker([row['lat'], row['lon']], popup=row['area']).add_to(m)

# 인터랙티브 맵 표시
m

In [87]:
# 테이블 병합하기
df = pd.merge(sample, location, how='left', left_on='MSRSTE_NM', right_on='area')
df.head(2)

,MSRDT,MSRRGN_NM,MSRSTE_NM,PM10,PM25,O3,NO2,CO,SO2,IDEX_NM,IDEX_MVL,ARPLT_MAIN,code,area,lon,lat
0,202308281400,도심권,중구,13.0,10.0,0.026,0.021,0.4,0.002,좋음,41.0,O3,11140,중구,126.995968,37.560144
1,202308281400,도심권,종로구,12.0,8.0,0.029,0.015,0.4,0.003,좋음,47.0,O3,11110,종로구,126.977321,37.594917


In [89]:
# 초기 맵 설정
# 각 자치구의 중심 좌표로 설정
m = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], 
               zoom_start=11)

# 데이터 포인트 추가
for i, row in df.iterrows():
    folium.Marker(
        [row['lat'], row['lon']], 
        popup=folium.Popup(f"<strong>{row['area']}</strong><br>초미세먼지: {row['PM10']}<br>미세먼지: {row['PM25']}", min_width=100, max_width=200),
        ).add_to(m)
    
# 인터랙티브 맵 표시
m

### .json 파일은 raw data로 잘 존재하지 않는다.
<br>국가공간정보포털<br>에서 .shp파일을 다운받아 별도의 라이브러리 통해 geo json 파일로 변경해주어야한다.

In [91]:
# 서울 자치구 GeoJSON 데이터
# https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json
import json
with open("seoul.json", encoding="utf-8") as file:
    geo_data = json.loads(file.read())
    
geo_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.11879551821994, 37.557222485451305],
      [127.12146867175024, 37.55986003393365],
      [127.12435254630417, 37.56144246249796],
      [127.13593925898998, 37.56564793048277],
      [127.14930548011061, 37.56892250303897],
      [127.15511020940411, 37.57093642128295],
      [127.16683184366129, 37.57672487388627],
      [127.17038810813094, 37.576465605301046],
      [127.17607118428914, 37.57678573961056],
      [127.17905504160184, 37.57791388161732],
      [127.17747787800164, 37.57448983055031],
      [127.1781775408844, 37.571481967974336],
      [127.17995281860672, 37.569309661290504],
      [127.18122821955262, 37.56636089217979],
      [127.18169407550688, 37.56286338914073],
      [127.18408792

In [92]:
from folium.plugins import MarkerCluster

m = folium.Map(
    location=[37.55, 126.96],
    tiles='cartodbpositron',    
    zoom_start=11, 
)

folium.GeoJson(
    geo_data,
    name='지역구',
).add_to(m)


marker_cluster = MarkerCluster().add_to(m)
for i, row in df.iterrows():
    folium.Marker(
        [row['lat'], row['lon']], 
        icon = folium.Icon(color="green"), 
        popup=folium.Popup(f"<strong>{row['area']}</strong><br>초미세먼지: {row['PM10']}<br>미세먼지: {row['PM25']}", 
                           min_width=100, 
                           max_width=200),
     ).add_to(marker_cluster)

m

In [86]:
m = folium.Map(
    location=[37.55, 126.96],
    tiles='cartodbpositron',    
    zoom_start=11, 
)

folium.GeoJson(
    geo_data,
    name='지역구',
).add_to(m)

m.choropleth(geo_data=geo_data,
             data=df.set_index('area')['PM25'], 
             fill_color='YlOrRd', 
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name="미세먼지"
            )
m

In [13]:
import json
sample_data = [1,2,3]

with open('./data/sample_data.json', 'w') as file:
    file.write(json.dumps(sample_data))

In [14]:
with open('.data/sample_data.json', 'r') as file:
    file_text = file.read()
    print(type(file_text))
    data = json.loads(file_text)
    print(type(data))

<class 'str'>
<class 'list'>
